In [1]:
import numpy as np
import re
from itertools import combinations, permutations
from copy import copy

In [2]:
from dataclasses import dataclass

@dataclass
class Cave():
    label: str
    tunnels: tuple[str]
    flow: int

In [3]:
def load_data(filename):
    with open(filename, 'r') as f:
        lines = map(str.strip, f.readlines())
    caves = []
    for l in lines:
        labels = re.findall('[A-Z]{2}', l)
        flow = int(re.search('\d+', l)[0])
        caves.append(Cave(labels[0], labels[1:], flow))
    return caves

In [4]:
def adjacency_matrix(nodes: list[Cave]) -> np.ndarray:
    index_mapping = {n.label: i for i, n in enumerate(nodes)}
    A = np.zeros((len(nodes), len(nodes)), dtype=int)
    for n in nodes:
        for t in n.tunnels:
            A[index_mapping[n.label], index_mapping[t]] = 1
    return A

In [5]:
# compute the distance matrix
def distance_matrix(A: np.ndarray) -> np.ndarray:
    """Uses the Floyd-Warshall algorithm to compute a distance
       matrix from an adjacency matrix.
       
    Args:
        A (np.ndarray): Adjacency matrix.
        
    Returns:
        np.ndarray
        
    """
    l = A.shape[0]
    D = np.where(A, A, 10000)
    d = np.diag([1]*l, 0)
    D = np.where(d, 0, D)
    for i, row in enumerate(D):
        neighbours = np.where(row != 10000)[0]
        for n1, n2 in permutations(neighbours, 2):
            d = min(row[n1] + row[n2], D[n1, n2])
            D[n1, n2] = d
            D[n2, n1] = d
    return D.astype(int)

In [6]:
class Path():
    
    def __init__(self,
                 time: int,
                 initial_cave: int) -> None:
        self.time = time
        self.visited=[initial_cave]
        self.pressure = 0
        
    def copy(self):
        new_path = Path(time=self.time,
                        initial_cave=self.visited[0])
        new_path.visited = self.visited.copy()
        new_path.pressure = self.pressure
        return new_path

In [7]:
filename = '../example_data/day16_example_data.txt'
caves = load_data(filename)
A = adjacency_matrix(caves)
D = distance_matrix(A)
labels = [c.label for c in caves]
flows = np.array([c.flow for c in caves])
initial_cave = next(i for i, c in enumerate(caves) if c.label == 'AA')
total_time = 30

path = Path(total_time, initial_cave)
stack = [path]
complete_paths = []
while stack:
    path = stack.pop(0)
    new = []
    possible_next_caves = [i for i, f in enumerate(flows) if (i not in path.visited) and (f != 0)]
    times_per_cave = [D[path.visited[-1]][c]+1 for c in possible_next_caves]
    for t, c in zip(times_per_cave, possible_next_caves):
        if path.time - t <= 0:
            continue
        extended_path = path.copy()
        extended_path.time -= t
        extended_path.visited.append(c)
        extended_path.pressure += (path.time - t) * flows[c]
        new.append(extended_path)
    if new:
        stack.extend(new)
    else:
        complete_paths.append(path)
max([p.pressure for p in complete_paths])

1651

In [8]:
filename = '../data/day16_data.txt'
caves = load_data(filename)
A = adjacency_matrix(caves)
D = distance_matrix(A)
labels = [c.label for c in caves]
flows = np.array([c.flow for c in caves])
initial_cave = next(i for i, c in enumerate(caves) if c.label == 'AA')
total_time = 30

path = Path(total_time, initial_cave)
stack = [path]
complete_paths = []
while stack:
    path = stack.pop(0)
    new = []
    possible_next_caves = sorted([i for i, f in enumerate(flows) if (i not in path.visited) and (f != 0)])
    times_per_cave = [D[path.visited[-1]][c]+1 for c in possible_next_caves]
    for t, c in zip(times_per_cave, possible_next_caves):
        if path.time - t <= 0:
            continue
        extended_path = Path(time=path.time - t, initial_cave=path.visited[0])
        extended_path.visited = path.visited + [c]
        extended_path.pressure = path.pressure + (path.time - t) * flows[c]
        new.append(extended_path)
    if new:
        stack.extend(new)
    else:
        complete_paths.append(path)

max([p.pressure for p in complete_paths])

1460